In [78]:
from pearce.emulator.emu import OriginalRecipe, ExtraCrispy
from pearce.emulator.trainingData import parameter, PARAMS

In [79]:
from george.kernels import ExpSquaredKernel
import george

In [80]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from itertools import cycle
colors = cycle(sns.color_palette())

In [81]:
training_dir = '/u/ki/swmclau2/des/PearceLHC/'

In [ ]:
or_params = PARAMS[:]
or_params.append(parameter('r', 0, 1)) #95% sure bounds aren't used for r
emu = OriginalRecipe(training_dir,or_params)

In [ ]:
ig = emu.get_initial_guess('xi', {})
a = ig['amp']
kernel = a * ExpSquaredKernel(emu.metric, ndim=emu.emulator_ndim)
loo_gp = george.GP(kernel)
# gp = george.GP(kernel, solver=george.HODLRSolver, nleaf=x.shape[0]+1,tol=1e-18)

loo_gp.compute(emu.x[1:], emu.yerr[1:])  # NOTE I'm using a modified version of george!

In [ ]:
K_inv_full = emu.gp.solver.apply_inverse(np.eye(emu.x.shape[0]),in_place=True)

In [ ]:
y_param = 'logMmin'
emulation_point = [('f_c', 0.233), ('logM0', 12.0), ('sigma_logM', 0.533), 
                   ('alpha', 1.083),('logM1', 13.5), ('logMmin', 12.233)]
em_params = dict(emulation_point)
del em_params[y_param]

In [ ]:
rbins = np.array([  0.06309573,   0.12437607,   0.24517359,   0.34422476, 0.48329302, 0.67854546,\
           0.9526807 , 1.33756775,1.8779508 ,   2.6366509 ,   3.70186906,   5.19743987, 7.29722764, \
           10.24533859,  14.38449888,  20.1958975 , 28.35512583,  39.81071706] )
rpoints= np.log10((rbins[1:]+rbins[:-1])/2)
em_params['r'] = rpoints[:2]

In [ ]:
y_param_idx = -1
for idx, p in enumerate(PARAMS):
    if p.name==y_param:
        y_param_idx = idx
        break
        
y_vals = np.linspace(PARAMS[y_param_idx].low, PARAMS[y_param_idx].high, 4)
em_params[y_param] = y_vals

In [ ]:
input_params = {}
input_params.update(em_params)

# i'd like to remove 'r'. possibly requiring a passed in param?
t_list = [input_params[p.name] for p in emu.ordered_params]
t_grid = np.meshgrid(*t_list)
t = np.stack(t_grid).T
t = t.reshape((-1, emu.emulator_ndim))
t = np.sort(t.view(','.join(['float64' for _ in xrange(min(t.shape))])),
            order=['f%d' % i for i in xrange(min(t.shape))], axis=0).view(np.float)

In [ ]:
loo_mu = loo_gp.predict(emu.y[1:], t , mean_only=True)
print loo_mu

In [ ]:
loo_inv = loo_gp.solver.apply_inverse(np.eye(emu.x.shape[0]-1),in_place=True)
print loo_inv.shape, K_inv_full.shape

In [ ]:
print loo_inv[0,:]
print 
print loo_inv[1,:]

In [ ]:
print K_inv_full[0,:]
print 
print K_inv_full[1,:]

In [ ]:
mu, _ = emu.emulate(em_params)
print mu

In [ ]:
yhat = loo_gp.predict(emu.y[1:], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)
print yhat

In [ ]:
print emu.y[0]

In [ ]:
x, y = emu.x[:], emu.y[:]

N = K_inv_full.shape[0]
mus = np.zeros((N, t.shape[0]))

In [ ]:
from time import time
from scipy.linalg import inv

In [ ]:
my_inds = [i for i in xrange(1, N-1)]
my_inds.append(0)
my_inds = np.array(my_inds)

In [ ]:
for idx in xrange(N):
    #print idx
    '''
    t0 = time()
    # This operation was really slow. Trying something else.
    # K_inv_idx_N = np.dot(rotation_matrix, np.dot(K_inv_full, rotation_matrix))
    # Small
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]
    #print time() - t0, 's'
    # swap x and y as well
    x[[idx, N - 1], :] = x[[N - 1, idx], :]
    y[[idx, N - 1]] = y[[N - 1, idx]]

    K_inv_m_idx = K_inv_full[:N - 1, :][:,
          :N - 1]  # -np.outer(K_inv_full[N-1,:N-1], K_inv_full[:N-1,N-1])/K_inv_full[N-1,N-1]
    c = K_inv_full[N - 1, :N - 1]
    # The subtracction takes a long time
    K_inv_m_idx -= np.outer(c / K_inv_full[N - 1, N - 1], c)
    Kxxs = emu.gp.kernel.value(t, x[:N - 1])
    #print time() - t0, 's'  # slow-ish
    # TODO currently doesn't work on ExtraCrispy. Could make y an input and it would.
    # this proudct takes a long time
    alpha = np.dot(K_inv_m_idx, y[:N - 1])
    #print time() - t0, 's'  # slow-ish
    mus[idx, :] = np.dot(Kxxs, alpha)

    print mus[idx, :]
    print
    #print time() - t0, 's'
    #print
    '''
    t0 = time()
    rotation_matrix = np.eye(N)
    rotation_matrix[idx, idx] = rotation_matrix[N - 1, N - 1] = 0.0
    rotation_matrix[idx, N - 1] = rotation_matrix[N - 1, idx] = 1.0
       
    #t_x = np.dot(rotation_matrix, emu.gp._x)
    #t_x  = np.dot(rotation_matrix, emu.x)
    print emu.x
    t_x = emu.x[:]
    print t_x
    t_x[[N-1, idx]] = t_x[[idx, N-1]]
    print t_x
    print np.dot(rotation_matrix, emu.x)
    print np.array_equal(t_x, np.dot(rotation_matrix, emu.x))
    t_yerr = np.dot(rotation_matrix, emu.yerr)
    #K3 = emu.gp.kernel.value(emu.gp._x, emu.gp._x)+np.diag(emu.gp._yerr**2)
    K3 = emu.gp.kernel.value(emu.x, emu.x)+np.diag(emu.yerr**2)
    K3 = np.dot(rotation_matrix, np.dot(K3, rotation_matrix))
    K3_inv = inv(K3[:N-1, :][:,:N-1])
    #y3 = np.dot(rotation_matrix, emu.gp._y)
    y3 = np.dot(rotation_matrix, emu.y)

    # i want to reproduce 4
    # 3 and 5 are the same
    #alpha3 = np.dot(K3_inv, y3[:N-1]-emu.gp.mean(t_x[:N-1]))
    #yhat3 = np.dot(K3[N-1,:N-1], alpha3)
    #print K3[N-1, :N-1]
    
    #same
    #means same point is being left out. 
    print 'x0'
    print emu.x[0]
    print t_x[N-1]
    print
    #these two have a different order for some reason! because ofc. 
    #do have the same values, just a different order
    print 'x'
    print loo_gp._x
    print t_x[:N-1]
    print t_x[:N-1][my_inds]
    print
    #similarly, same value with different ordering. 
    Kxxs4 = loo_gp.kernel.value(emu.x[0].reshape((1,emu.x.shape[1])), loo_gp._x)[0,:]
    print 'Kxx'
    print Kxxs4
    Kxxs5 = emu.gp.kernel.value(t_x[N-1].reshape((1,t_x.shape[1])), t_x[:N-1])[0,:]
    print Kxxs5
    print
    #also the same values, out of order. is the order different then of x?
    #no, just have to remember to use the gp's indicies
    print 'y'
    print loo_gp._y[loo_gp.inds]
    print y3[:N-1]
    print
    loo_gp._compute_alpha(loo_gp._y)
    alpha4 = loo_gp._alpha
    print alpha4
    #print
    alpha5 = np.dot(K3_inv, y3[:N-1]-emu.gp.mean(t_x[:N-1]))
    print alpha5
    print
    #print yhat3
    print np.dot(Kxxs4, alpha4) + loo_gp.mean(emu.x[0].reshape((1,emu.x.shape[1])))
    print yhat
    print np.dot(Kxxs5, alpha5)+ emu.gp.mean(t_x[N-1].reshape((1,t_x.shape[1])))
    print emu.gp._y[emu.gp.inds[0]] - emu.gp._alpha[0]/K_inv_full[0,0]
    print y3[N-1], emu.gp._y[0]
    #Kxxs3 =emu.gp.kernel.value(t, t_x[:N - 1])
    #mu3 = np.dot(Kxxs3, alpha3)
    #print mu3
    #print time() - t0, 's'
    '''
    K_inv_full[[idx, N - 1], :] = K_inv_full[[N - 1, idx], :]
    K_inv_full[:, [idx, N - 1]] = K_inv_full[:, [N - 1, idx]]
    #print time() - t0, 's'
    x[[idx, N - 1], :] = x[[N - 1, idx], :]
    y[[idx, N - 1]] = y[[N - 1, idx]]
    '''
    break

In [ ]:
print np.sum(np.array(sorted(Kxxs4))-np.array(sorted(Kxxs5)) )

In [ ]:
print np.sum(np.array(sorted(loo_gp._y))-np.array(sorted(y3[:N-1])) )

In [ ]:
idxs4 = np.argsort(loo_gp._y[loo_gp.inds])
idxs5 = np.argsort(y3[:N-1])
print loo_gp._y[loo_gp.inds][idxs4]
print
print y3[:N-1][idxs5]
print
print loo_gp._x[idxs4]
print 
print t_x[:N-1][idxs5]

In [ ]:
idxs = np.argsort(emu.y)
idxs2 = np.argsort(emu.gp._y)
print emu.y
print emu.gp._y[emu.gp.inds]
print
print emu.x
print emu.gp._x

In [ ]:
print np.sum(idxs-idxs2)

In [ ]:
print idxs4
print idxs5
#print sorted(idxs4- idxs5)
plt.plot(idxs4-idxs5)

In [ ]:
vals1 = {tuple(row) for row in loo_gp._x}
vals2 = {tuple(row) for row in t_x[:N-1]}
print vals1-vals2, len(vals1), len(vals2)

In [ ]:
plt.plot(sorted(alpha4), label = '4')
plt.plot(sorted(alpha5), label = '5')
plt.legend(loc='best')
plt.show()

In [ ]:
print loo_gp.predict(emu.y[1:], emu.x[0].reshape((1,emu.x.shape[1])), mean_only=True)

In [ ]:
print emu.x[0].reshape((1,emu.x.shape[1]))
print loo_gp.parse_samples(emu.x[0].reshape((1,emu.x.shape[1])))[0]
print
#print emu.x[1:]
Kxxs4 = loo_gp.kernel.value(emu.x[0].reshape((1,emu.x.shape[1])), emu.x[1:])[0,:]
print Kxxs4
print
loo_gp._compute_alpha(emu.y[1:])
alpha4 = loo_gp._alpha
print alpha4
print
print np.dot(Kxxs4, alpha4) + loo_gp.mean(emu.x[0].reshape((1,emu.x.shape[1])))
print 

In [ ]:
print emu.x[0].reshape((1,emu.x.shape[1]))
print loo_gp.parse_samples(emu.x[0].reshape((1,emu.x.shape[1])), True)[0]
print
#print loo_gp._x
Kxxs4 = loo_gp.kernel.value(emu.x[0].reshape((1,emu.x.shape[1])), loo_gp._x)[0,:]
print Kxxs4
print
loo_gp._compute_alpha(emu.y[1:])
alpha4 = loo_gp._alpha
print alpha4
print
print np.dot(Kxxs4, alpha4) + loo_gp.mean(emu.x[0].reshape((1,emu.x.shape[1])))
print 

In [ ]:
print loo_gp._x
print
print emu.x[1:]
print 
xx = loo_gp.parse_samples(emu.x[1:], True)[0]
print xx

In [ ]:
print loo_inv[0,:]
print 
print loo_inv[1,:]

In [ ]:
mu = mu.reshape((-1,2))
loo_mu = loo_mu.reshape((-1,2))
mu2 = mu2.reshape((-1,2))

In [ ]:
fig = plt.figure(figsize = (10, 10))

for label, lxp,looxp, lxp2, c in zip(y_vals, mu,loo_mu, mu2,colors):
    plt.plot(10**rpoints[:2], lxp,label = '%.3f'%label, color = c, marker ='o')
    plt.plot(10**rpoints[:2], looxp, color = c, marker ='v')
    plt.plot(10**rpoints[:2], lxp2, color = c, marker ='^')
    

#plt.xlim(xmin = 0.08, xmax = .6)    
#plt.ylim(ymin = 2, ymax = 4)
#plt.title("OR Emulator w.r.t. %s"%y_param, fontsize =20)
#plt.loglog()
plt.xscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel(r'$r $  $\rm{[Mpc]}$', fontsize=25)
plt.ylabel(r'$\log_{10}{\xi_{\rm gg}(r) }$', fontsize=25)
plt.legend(loc='best', fontsize=25)

In [ ]:
(N - 1) / N * np.cov(mus, rowvar=False)